In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import missingno as msno
from sklearn.impute import KNNImputer
# Autoreg, autocorrolationand time series tools...

from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
%matplotlib inline
from matplotlib import rcParams

from scipy.stats import skew

plt.style.use('seaborn-whitegrid')

from termcolor import colored

In [ ]:
city_day = pd.read_csv('/kaggle/input/air-quality-data-in-india/city_day.csv').sort_values(by = ['Date', 'City'])


In [ ]:
print(list(city_day.columns))

In [ ]:
city_day.Date = city_day.Date.apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
city_day = city_day.sort_values(by = 'Date')

In [ ]:
print('Date starts from {}, and ends in {}'.format(city_day.Date.min().strftime('%Y-%m-%d'), city_day.Date.max().strftime('%Y-%m-%d')))

In [ ]:
df=pd.read_csv('../input/air-quality-data-in-india/city_day.csv',parse_dates=['Date'])
df.head()


In [ ]:
df.describe()

In [ ]:
#here we calculate the proportion of null values in each of the features

#creating a func to make missing value table so that it can be used again
def missing_value_table(df):
    values = df.isnull().sum()
    percentage = 100*df.isnull().sum()/len(df)
    table = pd.concat([values,percentage.round(2)],axis=1)
    table.columns = ['No of missing values','% of missing values']
    return table[table['No of missing values']!=0].sort_values('% of missing values',ascending=False).style.background_gradient('Greens')
    
missing_value_table(pd.read_csv('../input/air-quality-data-in-india/city_day.csv'))

In [ ]:
city_day.corr().AQI.sort_values(ascending = False)

In [ ]:
pd.read_csv('/kaggle/input/air-quality-data-in-india/city_day.csv').isnull().sum().plot(kind ='bar')


In [ ]:
pd.read_csv('/kaggle/input/air-quality-data-in-india/station_day.csv').isnull().sum().plot(kind ='bar')

In [ ]:
df_city = pd.read_csv('/kaggle/input/air-quality-data-in-india/city_day.csv')
df_stations = pd.read_csv('/kaggle/input/air-quality-data-in-india/station_day.csv')
stations = pd.read_csv('../input/air-quality-data-in-india/stations.csv')
df_city['AQI'].fillna(method = 'ffill',inplace = True)
df_city['AQI'].fillna(method = 'bfill',inplace = True)
df_city['AQI'].fillna(value = 0,inplace = True)


df_stations['AQI'].fillna(method = 'ffill',inplace = True)
df_stations['AQI'].fillna(method = 'bfill',inplace = True)
df_stations['AQI'].fillna(value = 0,inplace = True)

In [ ]:
def missing_value_table(df):
    values = df.isnull().sum()
    percentage = 100*df.isnull().sum()/len(df)
    table = pd.concat([values,percentage.round(2)],axis=1)
    table.columns = ['No of missing values','% of missing values']
    return table[table['No of missing values']!=0].sort_values('% of missing values',ascending=False).style.background_gradient('Greens')
    
missing_value_table(pd.read_csv('../input/air-quality-data-in-india/city_day.csv'))

In [ ]:
most_polluted = city_day[['City', 'AQI', 'PM10', 'CO']].groupby(['City']).mean().sort_values(by = 'AQI', ascending = False)
most_polluted

In [ ]:
rcParams['axes.spines.top']=False
rcParams['axes.spines.right']=False


rcParams['figure.dpi']=300

rcParams['figure.autolayout']=True

rcParams['font.style']='normal'
rcParams['font.size']=4

rcParams['lines.linewidth']=0.7


rcParams['xtick.labelsize']=4
rcParams['ytick.labelsize']=4

In [ ]:
plt.style.use('seaborn-whitegrid')
f, ax_ = plt.subplots(1, 3, figsize = (15,15))

bar1 = sns.barplot(x = most_polluted.AQI,
                   y = most_polluted.index,
                   palette = 'Reds_r',
                   ax = ax_[0]);

bar1 = sns.barplot(x = most_polluted.PM10,
                   y = most_polluted.index,
                   palette = 'RdBu',
                   ax = ax_[1]);

bar1 = sns.barplot(x = most_polluted.CO,
                   y = most_polluted.index,
                   palette = 'RdBu',
                   ax = ax_[2]);

titles = ['AirQualityIndex', 'ParticulateMatter10', 'CO']
for i in range(3) :
    ax_[i].set_ylabel('')   
    ax_[i].set_yticklabels(labels = ax_[i].get_yticklabels(),fontsize = 14);
    ax_[i].set_title(titles[i])
    f.tight_layout()


In [ ]:
most_polluted = city_day[['City', 'AQI', 'PM10', 'CO']].groupby(['City']).mean().sort_values(by = 'AQI', ascending = False)

cities = most_polluted.index
params = most_polluted.columns

def first_date(city, parameter):
    df = city_day[(city_day.City == city)]
    df = df[df[parameter].notnull()]
    if len(df) != 0:
        return df.iloc[0].Date.strftime('%Y-%m-%d')
    else: return('no_measurement')
        
        
for city in cities:
    #print(colored('city: ', 'green'), city)
    for param in params:
      #  print('param: ', param)
        most_polluted.loc[city, str(param) + '_date'] = first_date(city, param)
        
most_polluted

In [ ]:
#segregating dates into months and years

df['Month']=df.Date.dt.month.astype(str)
df['Year']=df.Date.dt.year.astype(str)


In [ ]:
#line plot analysis for amount of particulate matter and gases  over the years
cols=['PM2.5','PM10','NO2','NOx','NH3',
      'CO','SO2','O3','Benzene','Toluene','Xylene']

x=df.iloc[:,2:]
fig=plt.figure(figsize=(3.2,6.5))
for i,col  in enumerate(cols):
    fig.add_subplot(6,2,i+1)
    sns.lineplot(x='Year',y=col,data=x)

In [ ]:
#distribuiton of various gases and particulate in air

cols=['PM2.5','PM10','NO2','NOx','NH3',
      'CO','SO2','O3','Benzene','Toluene','Xylene']

fig=plt.figure(figsize=(3,6.5))

for i,col in enumerate(cols):
    fig.add_subplot(6,2,i+1)
    plt.hist(df[col].dropna(),bins=20,edgecolor='white')
    plt.xlabel(col)
    plt.ylabel('Count')

In [ ]:
def tell_me_null(df):
    num_null = df.isnull().sum().sort_values(ascending = False)
    percentage_null = round(df.isnull().sum().sort_values(ascending = False)/len(df) * 100, 1)
    return pd.DataFrame(np.c_[num_null, percentage_null], index = num_null.index,  columns = ['# of Null', 'Percentage'])


In [ ]:
tell_me_null(city_day)

In [ ]:
delhi = city_day[(city_day.AQI.notnull()) & (city_day.City == 'Delhi')]

In [ ]:
delhi.corr().AQI.sort_values(ascending = False)

In [ ]:
tell_me_null(delhi)

In [ ]:
corr = delhi.corr().AQI.sort_values(ascending = False)
related = list(corr[corr>0.6].index)
print(related)
inter = delhi.loc[:, related].interpolate(method = 'linear');
delhi.loc[:, related] = inter

In [ ]:
delhi.columns

In [ ]:
# adding all the features with corr less than 0.4

city_day['B_X_O3_NH3'] = city_day['Benzene'] + city_day['Xylene'] + city_day['O3'] + city_day['NH3']

city_day['ParticulateMatters'] = city_day['PM2.5'] + city_day['PM10']

corr_with_AQI = city_day.corr().AQI.sort_values(ascending = False)


corr_with_AQI
knn_imputer = KNNImputer(n_neighbors = 3)

imputing_cols = [ 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI']
# we eliminated city, date, Year_Month and AQI_Bucket because 
# they either were unique or had numerical substitude in other fields(AQI_bucket)

knn_imputer.fit(delhi[imputing_cols])

imputed = knn_imputer.transform(delhi[imputing_cols])

delhi.loc[:, imputing_cols] = imputed

In [ ]:
tell_me_null(delhi)

In [ ]:
# we want to see what would have happened if there was no quarentine
# so we start our analysis using the data before 2020
# but first let's see how many records are in 2020

x_pred_after = delhi[delhi.Date > '2020-01-01']
delhi_b = delhi[delhi.Date < '2020-01-01']

print("the proportion of data recorded in 2020: {:.2f}".format (len(x_pred_after)/len(delhi)))

In [ ]:
#correlation analysis

plt.figure(figsize=(3,2))

mask=np.triu(df.corr(method='pearson'))
sns.heatmap(df.corr(method='pearson'),
            annot=True,fmt='0.1f',
            mask=mask,
            robust=True,
            cmap='pink')
plt.title('Correlation Analysis')

In [ ]:
#analyzing skewness in data
#here we observe high skewness in Bezene,Toluene,CO and Xylene

skew=pd.DataFrame(df.skew().sort_values(ascending=False))
skew.columns=['Skewness']

plt.figure(figsize=(2.5,1.8))
plt.xticks(rotation=75)
sns.barplot(x=skew.index,y='Skewness',data=skew,
            edgecolor='black',
            linewidth=0.2,
            palette='viridis_r')
plt.xlabel('Features')

In [ ]:
#plotting the average AQI over the years for top 5 cities which have AQI greater than 500

df['City'][df['AQI']>=500].value_counts()

cols=['Ahmedabad','Delhi','Patna','Gurugram','Lucknow']

for col in cols:
    plt.figure(figsize=(2,1.5))
    #selecting data pertaining to the selected city
    x=df[df['City']==col]
    sns.barplot(x='Year',y='AQI',data=x)
    plt.title(col)


In [ ]:
# Sum of pollution
import plotly.express as px

df = city_day.drop(columns = ['Date', 'AQI_Bucket', 'AQI']).groupby('City').sum().reset_index()
fig = px.treemap(pd.melt(df, id_vars = 'City'), path=['City','variable'],
                 values=pd.melt(df, id_vars = 'City')['value'],
                 title = 'Cities and the proportion of pollution in each')
fig.show()


In [ ]:
#cols represent particulate matter Carbon monoxide which have a correlation >=0.5 with AQI and thus 
#are responsible of increasing AQI of cities
cols=['PM2.5','PM10','CO','NO','NO2']

cmap=plt.get_cmap('Spectral')
color=[cmap(i) for i in np.linspace(0,1,8)]
explode=[0.2,0,0,0,0,0,0,0]

for col in cols:
    plt.figure(figsize=(2.8,1.8))
    
    '''grouping above columns by cities and 
    taking 8 cities which have the highest sum'''
    
    x=df.groupby('City')[col].sum().sort_values(ascending=False)
    x.reset_index('City')
    x[:8].plot.pie(shadow=True,autopct='%1.1f%%',
                   colors=color,explode=explode,
                   wedgeprops={'edgecolor':'black','linewidth':0.3}
                   )